# Funções

In [3]:
import time

#-----------------------------------------------------------------------------------
#     Função que plota o PPI das variáveis polarimétricas Z, ZDR, KDP e COR
#-----------------------------------------------------------------------------------
def quicklook_ppi_z_zdr_kdp_cor(fileout, radar):
        
    #========================================================================================================================#
    #                                               IMPORTA BIBLIOTECAS 
    #========================================================================================================================#
    import pyart 
    import matplotlib.pyplot as plt          
    import cartopy.crs as ccrs               
    
    #========================================================================================================================#
    #                                           EXTRAI INFORMAÇÕES DO RADAR
    #========================================================================================================================#
    # extrai a latitude e longitude do radar
    lat_radar, lon_radar = radar.latitude['data'][0], radar.longitude['data'][0]

    # limites dos dados
    latmin, latmax = radar.gate_latitude['data'].min(), radar.gate_latitude['data'].max()
    lonmin, lonmax = radar.gate_longitude['data'].min(), radar.gate_longitude['data'].max()

    # extrai a data do dado do radar
    data = pyart.util.datetime_from_grid(radar)

    # extrai ano, mês, dia, hora, minuto e segundos da data do radar
    ano = str(data.year)
    mes = str(data.month).zfill(2)
    dia = str(data.day).zfill(2)
    hor = str(data.hour).zfill(2)
    min = str(data.minute).zfill(2)
    seg = str(data.second).zfill(2)

    #========================================================================================================================#
    #                                                     PLOTA FIGURA
    #========================================================================================================================#
    # define a elevação
    level = 0 #(=primeira elevação)

    # extrai o ângulo de elevação da primeira elevação (level=0)
    str_level = str(radar.fixed_angle["data"][level])

    # moldura da figura
    fig = plt.figure(figsize=(9,6))

    # cria o objeto display
    display = pyart.graph.RadarMapDisplay(radar)

    # título de figura
    plt.suptitle(f'PPI {str_level}$^o$ {ano}-{mes}-{dia} {hor}:{min}:{seg} UTC', fontsize=13)

    #==========================#
    #       FIGURA 1: ZH
    #==========================#
    # moldura da figura
    ax1 = plt.subplot(221, projection=ccrs.PlateCarree())

    # plota o mapa
    display.plot_ppi_map('corrected_reflectivity',
                         sweep=level,
                         vmin=0,
                         vmax=70,
                         lat_0=lat_radar,
                         lon_0=lon_radar,
                         lat_lines=np.arange(latmin, latmax, 3),
                         lon_lines=np.arange(lonmin, lonmax, 3),
                         resolution='10m',
                         colorbar_label='Refletividade (dBZ)',
                         mask_outside=False,
                         ax=ax1,
                         cmap='pyart_NWSRef')

    # título da figura
    plt.title('ZH', fontsize=10)
    plt.title('a)', loc='left', fontsize=10)

    # plota aneis de distância
    display.plot_range_rings([100, 250], ax=ax1, col='gray', ls='--', lw=1)

    #==========================#
    #      FIGURA 2: ZDR
    #==========================#
    # moldura da figura
    ax2 = plt.subplot(222, projection=ccrs.PlateCarree())

    # plota ppi
    display.plot_ppi_map('corrected_differential_reflectivity',
                         sweep=level,
                         vmin=-2,
                         vmax=6,
                         lat_0=lat_radar,
                         lon_0=lon_radar,
                         lat_lines=np.arange(latmin, latmax, 3),
                         lon_lines=np.arange(lonmin, lonmax, 3),
                         resolution='10m',
                         colorbar_label='Refletividade Diferencial (dB)',
                         mask_outside=False,
                         ax=ax2)

    # título da figura
    plt.title('ZDR', fontsize=10)
    plt.title('b)', loc='left', fontsize=10)

    # plota aneis de distância
    display.plot_range_rings([100, 250], ax=ax2, col='gray', ls='--', lw=1)

    #==========================#
    #     FIGURA 3: KDP
    #==========================#
    # moldura da figura
    ax3 = plt.subplot(223, projection=ccrs.PlateCarree())

    # plota ppi
    display.plot_ppi_map('specific_differential_phase',
                         sweep=level,
                         vmin=-2,
                         vmax=6,
                         lat_0=lat_radar,
                         lon_0=lon_radar,
                         lat_lines=np.arange(latmin, latmax, 3),
                         lon_lines=np.arange(lonmin, lonmax, 3),
                         resolution='10m',
                         colorbar_label='Fase Diferencial Específica ($\degree$/km)',
                         mask_outside=False,
                         ax=ax3)

    # título da figura
    plt.title('KDP', fontsize=10)
    plt.title('c)', loc='left', fontsize=10)

    # plota aneis de distância
    display.plot_range_rings([100, 250], ax=ax3, col='gray', ls='--', lw=1)

    #==========================#
    #     FIGURA 4: COR
    #==========================#
    # moldura da figura
    ax4 = plt.subplot(224, projection=ccrs.PlateCarree())

    # plota ppi
    display.plot_ppi_map('cross_correlation_ratio',
                         sweep=level,
                         vmin=0.7,
                         vmax=1,
                         lat_0=lat_radar,
                         lon_0=lon_radar,
                         lat_lines=np.arange(latmin, latmax, 3),
                         lon_lines=np.arange(lonmin, lonmax, 3),
                         resolution='10m',
                         colorbar_label='Coeficiente de Correlação (#)',
                         mask_outside=False,
                         ax=ax4)

    # título da figura
    plt.title('RhoHV', fontsize=10)
    plt.title('d)', loc='left', fontsize=10)

    # plota aneis de distância
    display.plot_range_rings([100, 250], ax=ax4, col='gray', ls='--', lw=1)

    # recorta figura
    plt.tight_layout()

    # salva figura
    plt.savefig(fileout, dpi=300)
    print('...... Plotou quicklook \n')

# Processamento

In [ ]:
%%time
#========================================================================================================================#
#                                              IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pyart
import pandas as pd
import numpy as np
import glob
from datetime import datetime
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

#========================================================================================================================#
#                                                 DIRETÓRIOS
#========================================================================================================================#
dir_input = 'VOLUMETRICOS/'
dir_output = '/Users/enriq/Downloads/PROCESSAMENTO_PYTHON/ARTIGO_LITORAL_NORTE_2023/output/03_QUICKLOOK_PPI/'

#========================================================================================================================#
#                                                 LISTA DOS ARQUIVOS
#========================================================================================================================#
files = sorted(glob.glob(f'{dir_input}PNOVA2-2*HDF5')) # PNOVA2-20230218000002.HDF5
#files = files[0:1]

#========================================================================================================================#
#                                                   PROCESSAMENTO
#========================================================================================================================#
for file in files:

    print('PROCESSANDO ===>>', file)

    #--------------------------------------------------------------#
    #                     LEITURA DO DADO
    #--------------------------------------------------------------#
    # leitura do arquivo
    radar = pyart.aux_io.read_gamic(f'{file}')
    
    # extrai a data e horário do radar
    ano = str(pyart.util.datetime_from_grid(radar).year)
    mes = str(pyart.util.datetime_from_grid(radar).month).zfill(2)
    dia = str(pyart.util.datetime_from_grid(radar).day).zfill(2)
    hor = str(pyart.util.datetime_from_grid(radar).hour).zfill(2)
    min = str(pyart.util.datetime_from_grid(radar).minute).zfill(2)
    seg = str(pyart.util.datetime_from_grid(radar).second).zfill(2)

    # extrai a latitude, longitude e altitude do radar
    lat_radar, lon_radar = radar.latitude['data'][0], radar.longitude['data'][0]
    altitude = radar.altitude['data'][0]
    #variaveis = pd.DataFrame(list(radar.fields))
    #print(variaveis)
    print('...... 1. Extraiu informações da data')

    #--------------------------------------------------------------#
    #                   PLOTA QUICKLOOK DO PPI
    #--------------------------------------------------------------#
    quicklook_ppi_z_zdr_kdp_cor(f'{dir_output}quicklook_ppi_1elev_z_zdr_kdp_cor_fcth_{ano}{mes}{dia}_{hor}{min}{seg}.png', radar)

PROCESSANDO ===>> VOLUMETRICOS\PNOVA2-20230218000002.HDF5
...... 1. Extraiu informações da data
...... Plotou quicklook 

PROCESSANDO ===>> VOLUMETRICOS\PNOVA2-20230218000503.HDF5
...... 1. Extraiu informações da data
...... Plotou quicklook 

PROCESSANDO ===>> VOLUMETRICOS\PNOVA2-20230218001002.HDF5
...... 1. Extraiu informações da data
...... Plotou quicklook 

PROCESSANDO ===>> VOLUMETRICOS\PNOVA2-20230218001502.HDF5
...... 1. Extraiu informações da data
...... Plotou quicklook 

PROCESSANDO ===>> VOLUMETRICOS\PNOVA2-20230218002002.HDF5
...... 1. Extraiu informações da data
...... Plotou quicklook 

PROCESSANDO ===>> VOLUMETRICOS\PNOVA2-20230218002504.HDF5
...... 1. Extraiu informações da data
...... Plotou quicklook 

PROCESSANDO ===>> VOLUMETRICOS\PNOVA2-20230218003004.HDF5
...... 1. Extraiu informações da data
...... Plotou quicklook 

PROCESSANDO ===>> VOLUMETRICOS\PNOVA2-20230218003503.HDF5
...... 1. Extraiu informações da data
...... Plotou quicklook 

PROCESSANDO ===>> VOLUME